In [ ]:
import cplex
import numpy as np


B1 = np.array([[0.0, 0.0], [0.0, 0.0], [1.0,0.0], [0.0, 1.0]])*2.5
B2 = np.array([[0.0, 0.0], [0.0, 0.0], [1.0,0.0], [0.0, 1.0]])*1.666666667
d1 = np.array([[5000.0], [3500.0], [0.0], [0.0]])*2.5
d2 = np.array([ [5000.0], [5100.0], [0.0], [0.0]])*1.666666667
iteration = 0
ix = [[0.0], [0.0]]
zbar = 10000.0
zun = -10000.0
a = [0, 1, 2]
b = [0, 1, 3]
rhs2 = np.zeros(20)
pie1 = np.zeros((10, 4))
pie2 = np.zeros((10, 4))

while iteration <10:
    
    sub = cplex.Cplex()
        
    sub.objective.set_sense(sub.objective.sense.minimize)
    names = ["lsw", "lfw", "tpw", "tsw", "ssw", "sfw", "lsc", "lfc", "tpc", "tsc", "ssc", "sfc"]
    
    objective = [12.0, 29.20, 44.0, 44.0, -6.0, -26.0, 19.20, 43.80, 72.0, 72.0, -9.0, -39.0] 
    lower_bounds = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
    upper_bounds = [cplex.infinity, cplex.infinity, cplex.infinity, cplex.infinity, cplex.infinity, cplex.infinity, cplex.infinity, cplex.infinity, cplex.infinity, cplex.infinity, cplex.infinity, cplex.infinity]
        
    sub.variables.add(obj = objective,lb = lower_bounds,ub = upper_bounds,names = names)
    
    constraint_names = ["c1", "c2", "c3", "c4", "c5", "c6", "c7", "c8"]

    first_constraint = [["tpw", "tsw"], [1.0, 1.0]]

    second_constraint = [["tpw", "tsw"], [1.0, 1.20]]

    thirdd_constraint = [["tsw", "ssw", "lsw"], [1.0, 1.0, -1.0]]

    fourth_constraint = [["tpw", "tsw", "sfw", "lfw"], [1.0, 1.0, 1.0, -1.0]]

    fifth_constraint = [["tpc", "tsc"], [1.0, 1.0]]

    sixth_constraint = [["tpc", "tsc"], [1.0, 1.10]]

    seventh_constraint = [["tsc", "ssc", "lsc"], [1.0, 1.0, -1.0]]

    eighth_constraint = [["tpc", "tsc", "sfc", "lfc"], [1.0, 1.0, 1.0, -1.0]]

    constraints = [ first_constraint, second_constraint, thirdd_constraint, fourth_constraint, fifth_constraint, sixth_constraint, seventh_constraint, eighth_constraint]


    rhs = [5000.0, 3500.0, ix[0][0], ix[1][0], 5000.0, 5100.0, ix[0][0], ix[1][0]]


    constraint_senses = [ "L", "G", "E", "E", "L", "G", "E", "E"]


    sub.linear_constraints.add(lin_expr = constraints,
                               senses = constraint_senses,
                               rhs = rhs,
                               names = constraint_names)
    sub.solve()
    
    pie1[iteration] = sub.solution.get_dual_values([0, 1, 2, 3])
    pie2[iteration] = sub.solution.get_dual_values([4, 5, 6, 7])
    z = sub.solution.get_objective_value()
    zhat = z + 20*ix[0][0] + 70*ix[1][0]
    
    if zbar<zhat: 
        zbar = zhat
        

    

    master = cplex.Cplex() 
    master.objective.set_sense(master.objective.sense.minimize)
    names = ["xs", "xf", "teta1", "teta2"]
    objective = [20.0, 70.0, 0.4, 0.6] 
    lower_bounds = [0.0, 0.0, 0.0, 0.0]
    upper_bounds = [cplex.infinity, cplex.infinity, cplex.infinity, cplex.infinity]
    master.variables.add(obj = objective,lb = lower_bounds,ub = upper_bounds,names = names)
    
    
    cut = [[[a], [b]], [[a], [b]], [[a], [b]], [[a], [b]], [[a], [b]], [[a], [b]], [[a], [b]], [[a], [b]], [[a], [b]], [[a], [b]], [[a], [b]], [[a], [b]], [[a], [b]], [[a], [b]], [[a], [b]], [[a], [b]], [[a], [b]], [[a], [b]], [[a], [b]], [[a], [b]]]
            
    for i in range(0, 10):
        cut[i] = [a, [-pie1[i]@B1[:,0], -pie1[i]@B1[:,1], 1.0]]
    
    for i in range(10, 20):
        cut[i] = [b, [-pie2[i-10]@B2[:,0], -pie2[i-10]@B2[:,1], 1.0]]    
    
    for i in range(0,10):
        rhs2[i] = pie1[i]@d1
    
    for i in range(10,20):
        rhs2[i] = pie2[i-10]@d2
          
    master.linear_constraints.add(lin_expr = cut,
                               senses = "GGGGGGGGGGGGGGGGGGGG",
                               rhs = rhs2)
                
    master.solve()    
    zun = master.solution.get_objective_value()
    err = (zbar - zun)/min(abs(zbar),abs(zun))
    print("in Iteration", iteration,"[xs, xf, teta1, teta2] = ", master.solution.get_values())
    print("in Iteration", iteration, "[zbar, zhat, zun, err] = ",[zbar, zhat, zun, err])
    ix = [[master.solution.get_values(0)], [master.solution.get_values(1)]]
    
    iteration += 1

    
    
            
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
            
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    